#  Display options:

In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph_BB"

print("--")


--


In [4]:

#  Create database

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


4MAP8VHNkHLJQX6VhgBzqgpubGj5ot8c84iWJxY2vG2p


In [5]:

#  Create graph
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph_BB, EDJ39S7aoeT2aT2BchYPHzwHB14JUfCLdZ2HKAS6eDqn, 0>


In [ ]:

#  Connect to graph

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


In [6]:

#  Now that we share servers, get a better list of databases and graphs ..
#

for l_each_d in my_client.databases():
   print("Database Name: %-28s   Id: %s" % (l_each_d.name, l_each_d.database_id))
      #
   for l_each_g in my_client.get_database(name=l_each_d.name).graphs_in_database(): 
      print("   Graph Name: %-28s      Version: %-8d   Id: %s" % (l_each_g.name, l_each_g.version, l_each_g.graph_id))


print(""  )
print("--")


Database Name: my_db                          Id: 4MAP8VHNkHLJQX6VhgBzqgpubGj5ot8c84iWJxY2vG2p
   Graph Name: my_graph_BB                       Version: 0          Id: EDJ39S7aoeT2aT2BchYPHzwHB14JUfCLdZ2HKAS6eDqn
Database Name: default                        Id: QnXF6oF6UFgJhbwqvu7VxmSprvKxMakG3wkMW5jMFqg

--


In [ ]:

from ipywidgets import widgets
from ipywidgets import Box

l_dbname   = widgets.Text(
    value        = "  < paste DB-Name here >",
    description  = "DB Name: ",
    disabled     = False
)
l_graphname = widgets.Text(
    value        = "  < paste Graph-Id here >",
    description  = "Graph Name: ",
    disabled     = False
)

l_items = [l_dbname, l_graphname]
l_box = Box(children=l_items)

l_box


In [ ]:

#  Delete database by name
#

for l_database in my_client.databases():
   if (l_database.name != "default"):
      if (l_database.name == l_dbname.value):
         try:
            my_client.get_database(name=l_database.name).delete_database()
            print("Database deleted: %s" % (l_database.name))
         except:
            print("ERROR:  Database not deleted.  The most likely cause is that the named database is not empty.")

print("")
print("Existing databases ..")
   #
for l_database in my_client.databases():
   print("   DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


print(""  )
print("--")



In [ ]:

#  Delete graph by name
#

for l_each_d in my_client.databases():
   for l_each_g in my_client.get_database(name = l_each_d.name).graphs_in_database(): 
      #  if (l_each_g.graph_id == l_graphid.value):
      if (l_each_g.name == l_graphid.value):
         try:
            l_handle = my_client.get_database(name = l_each_d.name).get_graph_by_id(id = l_each_g.graph_id)
            l_handle.delete()
            print("Graph deleted: %s" % (l_each_g.name))
         except:
            print("ERROR:  Graph not deleted.  The most likely cause is unknown.")

            
print("")
print("Existing databases and graphs ..")
   #
for l_each_d in my_client.databases():
   print("   Database Name: %-28s   Id: %s" % (l_each_d.name, l_each_d.database_id))
      #
   for l_each_g in my_client.get_database(name=l_each_d.name).graphs_in_database(): 
      print("      Graph Name: %-28s      Version: %-8d   Id: %s" % (l_each_g.name, l_each_g.version, l_each_g.graph_id))


print(""  )
print("--")



In [ ]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


#  UDFs, Part 01: Just MPI

In [ ]:

#  Best practice; Have a second NoteBook open with the Operations Widget ready-
#
#  Why ?
#     When you make mistakes below, you will want to kill the Python job that is running
#     on the worker nodes, the UDFs you submit below-
#
#  Source,
#     https://mpi4py.readthedocs.io/en/stable/tutorial.html


In [ ]:

#  Example for testing; generates a random dataset
#

import time
   #
from IPython.display import clear_output


def my_func():
    
   from katana.distributed import single_host
      #
   import numpy as np                                                     #  Unparitioned objects are just fine; we are working from only
                                                                          #  one node
   l_return = np.random.randint(1, 101, 4)                                #  Generate an array[4] of random numbers in the range of 1-100

   return single_host(host=0, result=l_return)                            #  Multiple worker nodes (hosts), can only return from 1


l_cntr = 0
   #
while True:
   l_cntr += 1
   print("Running interation:  %-16d" % (l_cntr))
      #
   my_return = my_graph.run(lambda g: my_func())
   time.sleep(60) 
   clear_output(wait = True)
    


Running interation:  315             


          0/? [?op/s]